In [2]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import scipy.sparse
import joblib
from collections import Counter
import re
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


False

In [3]:
ori_df = pd.read_json("scicite/train.jsonl", lines=True)
df = pd.read_json("scicite/train.jsonl", lines=True)

df.shape

(8243, 15)

In [4]:
df = df.drop(columns=['label_confidence','citingPaperId','citedPaperId','id','unique_id','label2','label2_confidence'])

df.shape

(8243, 8)

In [5]:
for column in df.columns:
    print(f"{column} null value:",sum(df[column].isnull()))

source null value: 2
citeEnd null value: 2
sectionName null value: 19
citeStart null value: 2
string null value: 0
label null value: 0
isKeyCitation null value: 0
excerpt_index null value: 0


In [6]:
df = df.dropna(subset=['source', 'citeEnd', 'citeStart'])
display(
    f"Number of null values in 'source': {sum(df['source'].isnull())}",
    f"Number of null values in 'citeEnd': {sum(df['citeEnd'].isnull())}",
    f"Number of null values in 'citeStart': {sum(df['citeStart'].isnull())}"
)

"Number of null values in 'source': 0"

"Number of null values in 'citeEnd': 0"

"Number of null values in 'citeStart': 0"

In [7]:
# Initialize OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit and transform the 'source' column
one_hot_encoded_source = one_hot_encoder.fit_transform(df[['source']])

# Get new column names for 'source'
one_hot_columns_source = one_hot_encoder.get_feature_names_out(['source'])

# Create a DataFrame with one-hot encoded values for 'source'
df_one_hot_source = pd.DataFrame(one_hot_encoded_source, columns=one_hot_columns_source, index=df.index)

# Fit and transform the 'isKeyCitation' column
one_hot_encoded_key_citation = one_hot_encoder.fit_transform(df[['isKeyCitation']])

# Get new column names for 'isKeyCitation'
one_hot_columns_key_citation = one_hot_encoder.get_feature_names_out(['isKeyCitation'])

# Create a DataFrame with one-hot encoded values for 'isKeyCitation'
df_one_hot_key_citation = pd.DataFrame(one_hot_encoded_key_citation, columns=one_hot_columns_key_citation, index=df.index)

# Concatenate the one-hot encoded DataFrames with the original DataFrame
df = pd.concat([df, df_one_hot_source, df_one_hot_key_citation], axis=1)

# Drop the original categorical columns if not needed
df.drop(columns=['source', 'isKeyCitation'], inplace=True)

# Show one-hot encoded DataFrame and the new shape
print("Source Mapping:", one_hot_columns_source)
print("Key Citation Mapping:", one_hot_columns_key_citation)
df.shape

Source Mapping: ['source_acronym' 'source_acronymParen' 'source_andPhrase'
 'source_etAlPhrase' 'source_explicit' 'source_properNoun']
Key Citation Mapping: ['isKeyCitation_False' 'isKeyCitation_True']


(8241, 14)

In [8]:
# convert float to int
df.loc[:, 'citeStart'] = df['citeStart'].astype(np.int64)
df.loc[:,'citeEnd'] = df['citeEnd'].astype(np.int64)

In [9]:
# replace NaN and empty strings with "Unknown"
df['sectionName'] = df['sectionName'].replace('', np.nan).fillna("Unknown")
sum(df['sectionName'].isnull())

0

In [10]:
df.loc[:, 'sectionName'] = df['sectionName'].apply(str.lower)
len(df['sectionName'].unique())

1074

In [11]:
# Remove numbers using regex
df['sectionName'] = df['sectionName'].str.replace(r'\d+', '', regex=True).str.strip()
len(df['sectionName'].unique())

863

In [12]:
df['sectionName'] = df['sectionName'].str.replace(r'[\d{}]+'.format(string.punctuation), '', regex=True).str.strip()
len(df['sectionName'].unique())

807

In [13]:
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to lemmatize text
def lemmatize_text(text):
    words = nltk.word_tokenize(text)  # Tokenize the text into words
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatize each word
    return " ".join(lemmatized_words)

# Apply lemmatization to the 'sectionName' column
df['sectionName'] = df['sectionName'].apply(lemmatize_text)
len(df['sectionName'].unique())

794

In [14]:
# List of words to check for in the 'sectionName' column
keywords = ['methodology', 'method', 'approach', 'design', \
            'proposed', 'experiment', 'implementation', 'procedure']

# Function to replace the entire string with 'Method' if any keyword is found
def replace_with_method(text):
    if any(keyword in text.lower() for keyword in keywords):  # Check if any keyword is present
        return 'method'
    return text

# Apply the function to the 'sectionName' column
df['sectionName'] = df['sectionName'].apply(replace_with_method)

len(df['sectionName'].unique())

732

In [15]:
# List of words to check for in the 'sectionName' column
keywords = ['result', 'discussion', 'evaluation', 'limitation', 'implication', 'comparison']

# Function to replace the entire string with 'Method' if any keyword is found
def replace_with_method(text):
    if any(keyword in text.lower() for keyword in keywords):  # Check if any keyword is present
        return 'result'
    return text

# Apply the function to the 'sectionName' column
df['sectionName'] = df['sectionName'].apply(replace_with_method)

len(df['sectionName'].unique())

686

In [16]:
# List of words to check for in the 'sectionName' column
keywords = ['background', 'introduction', 'related work', 'relatedwork', 'overview']

# Function to replace the entire string with 'Method' if any keyword is found
def replace_with_method(text):
    if any(keyword in text.lower() for keyword in keywords):  # Check if any keyword is present
        return 'background'
    return text

# Apply the function to the 'sectionName' column
df['sectionName'] = df['sectionName'].apply(replace_with_method)

len(df['sectionName'].unique())

665

In [17]:
# List of words to check for in the 'sectionName' column
keywords = ['conclusion', 'concluding', 'future work']

# Function to replace the entire string with 'Method' if any keyword is found
def replace_with_method(text):
    if any(keyword in text.lower() for keyword in keywords):  # Check if any keyword is present
        return 'conclusion'
    return text

# Apply the function to the 'sectionName' column
df['sectionName'] = df['sectionName'].apply(replace_with_method)

len(df['sectionName'].unique())

659

In [116]:
df['sectionName'].unique()

array(['background',
       'novel quantitative trait locus for seminal root trait in barley',
       'result', 'method', 'providing selfdetermination opportunity',
       'mitochondrial lesion', 'unknown', 'go cellular component',
       'economic and policy environment', 'scope and extension', 'vglut',
       'technical consideration', 'pulse wave velocity',
       'interpretation bias in social anxiety', 'accuracy',
       'actinobacteria', 'cryostat',
       'c uplink capacity analysis for massive mimo', 'ata collection',
       'present address', 'no o', 'multiple kernel learning',
       'node description', 'q protein of phage λ', 'conclusion',
       'geneseer search feature', 'virus and cell',
       'empirical kcef and consistency',
       'a is well known this concept can be expressed equivalently using a relation between set of proposition and individual proposition instead of an operation for example with the following',
       'pharmacokinetic analysis',
       'molecular 